In [1]:
import json
import re
import urllib.request

from bs4 import BeautifulSoup
import pandas as pd

In [7]:
ROCHADE_URL = "https://rochadeeuropa.com/lichess-turniere-beendet/"
BUNDESLIGA_REGEXP = r"|".join(["(1. DE-Quarantäne Team Battle)",
                               "([0-9]+\. ?DE[ -]Quarantäne Teams 1-10)",
                               "(5. Quarantäne-Liga Teams 1-10)",
                               "([0-9]+\. ?Quarantäne-Bundesliga)",
                               "([0-9]+\. ?Quarantäne-Welt-Bundesliga ?$)",
                               ])


In [8]:
def get_bundesliga_tournaments():
    """
    scrape rochadeeuropa.com in order to find lichess quarantaine bundesliga matches.
    Rochade URL and regex to determine which tournament was actually a bundesliga tournaments are 
    taken from configurable global variables
    """
    # scrape rochade finished lichess tournaments
    response = urllib.request.urlopen(ROCHADE_URL)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    # parse table
    data = []
    table = soup.find('table', attrs={'class':'tablepress tablepress-id-3'})
    table_body = table.find('tbody')
    
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data.append([ele.text.strip() for ele in cols])
    # filter for bundesliga
    buli_tournaments = [el for el in data if re.match(BUNDESLIGA_REGEXP, el[1])]
    return buli_tournaments

In [9]:
def get_individual_results(tournament_id):
    """
    For individual results for a given tournament we use lichess API
    """
    api_url = f"https://lichess.org/api/tournament/{tournament_id}/results"
    api_response = urllib.request.urlopen(api_url)
    player_results = api_response.read()
    return pd.DataFrame([json.loads(pl) for pl in player_results.splitlines()])

In [10]:
def get_team_results(tournament_url):
    """
    Unfortunately the API does not provide team results, hence we need to go the pedestrian way and 
    scrape it by hand
    """
    tournament_response = urllib.request.urlopen(tournament_url)
    tournament_html = tournament_response.read()
    tournament_soup = BeautifulSoup(tournament_html, 'html.parser')
    # there is a string before the json that needs to be deleted.
    # This seems a little clumsy and error prone but there is no other way when parsing manually
    str_to_delete = "lichess.tournament="
    tournament_strings = [script.text for script in tournament_soup.find_all("script") 
                          if str_to_delete in script.text]
    tournament_strings.sort(key=lambda s: len(s))
    tournament_string = tournament_strings[-1]
    tournament_json = json.loads(tournament_string[tournament_string.find(str_to_delete)+len(str_to_delete):])
    team_name_dict = tournament_json["data"]["teamBattle"]["teams"]
    relevant_keys = ["rank", "id", "score"]
    team_results = [{k: v for k, v in team_dict.items() if k in relevant_keys} for team_dict in 
                    tournament_json["data"]["teamStanding"]]
    teams_df = pd.DataFrame(team_results)
    teams_df["Team"] = teams_df["id"].map(team_name_dict)
    return teams_df

In [11]:
team_df = pd.DataFrame()
individual_df = pd.DataFrame()

buli_tournaments = get_bundesliga_tournaments()
for tournament in buli_tournaments:
    print(f"Downloading data of tournament {tournament[1]} on {tournament[0]}")
    team_df = team_df.append(get_team_results(tournament[4]))
    individual_df = individual_df.append(get_individual_results(tournament[4].split("/")[-1]))
    
team_df["count"] = 1
team_df["champion"] = (team_df["rank"] == 1).astype(int)
individual_df["count"] = 1
individual_df["champion"] = (individual_df["rank"] == 1).astype(int)

In [12]:
all_time_teams = team_df.groupby("Team", as_index=False).agg(
    {"score": "sum", "count": "count", "champion": "sum", "rank": "mean"}).sort_values("score", ascending=False)
all_time_teams.columns = ["Team", "Gesamtpunkte", "Teilnahmen", "Meisterschaften", "Durchschnittsplatzierung"]
all_time_teams.index = range(1, len(all_time_teams) + 1)
all_time_indiv = individual_df.groupby("username", as_index=False).agg(
    {"score": "sum", "count": "count", "champion": "sum", "performance": "mean", "rank": "mean", }).sort_values("score", ascending=False)
all_time_indiv.columns = ["Name", "Gesamtpunkte", "Teilnahmen", "Turniersiege", "Durchschnittsperformance", "Durchschnittsplatzierung"]
all_time_indiv.index = range(1, len(all_time_indiv) + 1)
all_time_indiv["Durchschnittsscore"] = all_time_indiv["Gesamtpunkte"] / all_time_indiv["Teilnahmen"]

In [13]:
all_time_teams.reset_index().rename(columns={"index": "Rang"})

,Rang,Team,Gesamtpunkte,Teilnahmen,Meisterschaften,Durchschnittsplatzierung
0,1,Bierstube MUC,5691,20,3,3.300000
1,2,HSK Lister Turm,4767,16,4,4.125000
2,3,KSK Dr. Lasker 1861 e.V.,3840,13,1,5.153846
3,4,Oxfordgang,3044,13,0,6.000000
4,5,Delfintherapie,2528,7,3,3.857143
5,6,Sfr. Heidesheim,2290,7,1,6.142857
6,7,SK Landau 1908 e.V.,2254,8,0,5.875000
7,8,TSV Schöniach,2226,9,0,5.222222
8,9,Die Nordlichter,1859,9,3,5.111111
9,10,Wiener Kandidaten und Fake Meister,1859,7,0,6.285714


In [16]:
all_time_indiv.round({"Durchschnittsplatzierung": 1, "Durchschnittsperformance": 1, "Durchschnittsscore": 1})

,Name,Gesamtpunkte,Teilnahmen,Turniersiege,Durchschnittsperformance,Durchschnittsplatzierung,Durchschnittsscore
1,SomeMightSay,1132,20,5,2596.1,4.6,56.6
2,psammenitos,714,14,4,2586.8,15.1,51.0
3,Kabeljaukrieger,668,19,0,2394.9,33.3,35.2
4,Satscho,648,20,0,2348.8,42.3,32.4
5,Eldrail,610,16,0,2478.8,29.9,38.1
6,P1W4,568,20,0,2170.9,62.0,28.4
7,fluffy_cactus,536,20,0,2222.8,67.8,26.8
8,Donnerhall,524,13,0,2521.4,27.7,40.3
9,Chillkroete77,519,13,1,2370.1,16.1,39.9
10,Unkreativ3,511,15,0,2452.6,40.5,34.1


In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def connect_to_spreadsheet():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']

    credentials = ServiceAccountCredentials.from_json_keyfile_name('/home/paul/Downloads/dummyproject-273213-6c2b9c180973.json', scope)

    gc = gspread.authorize(credentials)
    return gc

In [11]:
import gspread_dataframe as gd

def write_to_spreadsheet(df, sheet_connection):
    ws = sheet_connection.open("Ewige Quarantäne-Bundesligatabelle").worksheet("Sheet1")
    gd.set_with_dataframe(ws, df)

In [12]:
df_to_write = all_time_teams.reset_index().rename(columns={"index": "Rang"}).round({"Durchschnittsplatzierung": 1})

In [13]:
conn = connect_to_spreadsheet()

In [14]:
write_to_spreadsheet(df_to_write, conn)